In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
%tensorflow_version 1.x
!pip install texar

In [0]:
import tensorflow as tf
import texar.tf as tx

import numpy as np
import pprint
import logging

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_paths, params):
  char2idx = params['char2idx']
  for f_path in f_paths:
    with open(f_path) as f:
      print('Reading', f_path)
      for line in f:
        line = line.rstrip()
        sp = line.split('|')
        if len(sp) == 2:
          source, target = sp
        else:
          continue
        source = [char2idx.get(c, len(char2idx)) for c in list(source)]
        target = [char2idx.get(c, len(char2idx)) for c in list(target)]
        if len(source) > params['max_len']:
          source = source[:params['max_len']]
        if len(target) > params['max_len']:
          target = target[:params['max_len']]
        target_in = [1] + target
        target_out = target + [2]
        if len(source) > 0:
          yield (source, (target_in, target_out))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ([None], [None]))
  _types = (tf.int32, (tf.int32, tf.int32))
  _pads = (0, (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)


def rnn_cell():
    def cell_fn():
        cell = tf.nn.rnn_cell.LSTMCell(params['rnn_units'],
                                       initializer=tf.orthogonal_initializer())
        return cell
    if params['dec_layers'] > 1:
      cells = []
      for i in range(params['dec_layers']):
        if i == params['dec_layers'] - 1:
          cells.append(cell_fn())
        else:
          cells.append(tf.nn.rnn_cell.ResidualWrapper(cell_fn(), residual_fn=lambda i,o: tf.concat((i,o), -1)))
      return tf.nn.rnn_cell.MultiRNNCell(cells)
    else:
      return cell_fn()

  
def dec_cell(enc_out, enc_seq_len):
    attn = tf.contrib.seq2seq.BahdanauAttention(
        num_units = params['rnn_units'],
        memory = enc_out,
        memory_sequence_length = enc_seq_len)
    
    return tf.contrib.seq2seq.AttentionWrapper(
        cell = rnn_cell(),
        attention_mechanism = attn,
        attention_layer_size = params['rnn_units'])

In [0]:
class TiedDense(tf.layers.Layer):
  def __init__(self, tied_embed, out_dim):
    super().__init__()
    self.tied_embed = tied_embed
    self.out_dim = out_dim
  
  def build(self, input_shape):
    self.bias = self.add_weight(name='bias',
                                shape=[self.out_dim],
                                trainable=True)
    if params['rnn_units'] != 300:
      self.proj_W = self.add_weight(name='proj_W',
                                    shape=[params['rnn_units'], params['embed_dim']],
                                    trainable=True)
      self.proj_b = self.add_weight(name='proj_b',
                                    shape=[params['embed_dim']],
                                    trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    if params['rnn_units'] != 300:
      inputs = params['activation'](tf.nn.bias_add(tf.matmul(inputs, self.proj_W), self.proj_b))
    x = tf.matmul(inputs, self.tied_embed, transpose_b=True)
    x = tf.nn.bias_add(x, self.bias)
    return x
  
  def compute_output_shape(self, input_shape):
    return input_shape[:-1].concatenate(self.out_dim)

In [0]:
def sampling_proba_fn():
  global_step = tf.train.get_or_create_global_step()
  return 1 - tf.maximum(tf.train.exponential_decay(.95, global_step, 1000, .99), .5)

In [0]:
def forward(words, labels, mode):
    words_len = tf.count_nonzero(words, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(words)[0]
    
    
    with tf.variable_scope('Embedding'):
        embedding = tf.Variable(np.load('../vocab/char.npy'),
                                dtype=tf.float32,
                                name='fasttext_vectors')
        x = tf.nn.embedding_lookup(embedding, words)
        x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
    
    
    with tf.variable_scope('Encoder'):
        t = tf.transpose(x, perm=[1, 0, 2])  # Need time-major
        lstm_cell_fw = tf.contrib.rnn.LSTMBlockFusedCell(params['rnn_units'])
        lstm_cell_bw = tf.contrib.rnn.LSTMBlockFusedCell(params['rnn_units'])
        lstm_cell_bw = tf.contrib.rnn.TimeReversedFusedRNN(lstm_cell_bw)
        o_fw, s_fw = lstm_cell_fw(t, dtype=tf.float32, sequence_length=words_len)
        o_bw, s_bw = lstm_cell_bw(t, dtype=tf.float32, sequence_length=words_len)
        enc_out = tf.concat([o_fw, o_bw], axis=-1)
        enc_out = tf.transpose(enc_out, perm=[1, 0, 2])

        enc_state = tf.layers.dense(tf.concat((s_fw.h, s_bw.h), -1), params['rnn_units'], params['activation'], name='state_fc')
        enc_state = tf.nn.rnn_cell.LSTMStateTuple(c=enc_state, h=enc_state)
        if params['dec_layers'] > 1:
          enc_state = tuple(params['dec_layers'] * [enc_state])
    
    
    with tf.variable_scope('Decoder'):
        output_proj = TiedDense(embedding, len(params['char2idx'])+1)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            dec_inputs, dec_outputs = labels
            dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
            dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
            dec_inputs = tf.layers.dropout(dec_inputs, params['dropout_rate'], training=is_training)
            cell = dec_cell(enc_out, words_len)
            
            init_state = cell.zero_state(batch_sz, tf.float32).clone(
                cell_state=enc_state)
            
            if is_training:
              helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
                  inputs = dec_inputs,
                  sequence_length = dec_seq_len,
                  embedding = embedding,
                  sampling_probability = sampling_proba_fn(),)
            else:
              helper = tf.contrib.seq2seq.TrainingHelper(
                  inputs = dec_inputs,
                  sequence_length = dec_seq_len,)
            decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = cell,
                helper = helper,
                initial_state = init_state,
                output_layer = output_proj)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = tf.reduce_max(dec_seq_len))
            
            return decoder_output.rnn_output
        else:
            enc_out_t = tf.contrib.seq2seq.tile_batch(enc_out, params['beam_width'])
            enc_state_t = tf.contrib.seq2seq.tile_batch(enc_state, params['beam_width'])
            enc_seq_len_t = tf.contrib.seq2seq.tile_batch(words_len, params['beam_width'])
            
            cell = dec_cell(enc_out_t, enc_seq_len_t)
            
            init_state = cell.zero_state(batch_sz*params['beam_width'], tf.float32).clone(
                cell_state=enc_state_t)
            
            decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = cell,
                embedding = embedding,
                start_tokens = tf.tile(tf.constant([1], tf.int32), [batch_sz]),
                end_token = 2,
                initial_state = init_state,
                beam_width = params['beam_width'],
                output_layer = output_proj,
                length_penalty_weight = params['length_penalty'],
                coverage_penalty_weight = params['coverage_penalty'],)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = params['max_len'],)
            
            return decoder_output.predicted_ids[:, :, :params['top_k']]

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    if (params['label_smoothing'] <= .0) or (mode == tf.estimator.ModeKeys.EVAL):
      loss_op = tf.contrib.seq2seq.sequence_loss(logits = logits_or_ids,
                                                 targets = dec_outputs,
                                                 weights = tf.to_float(tf.sign(dec_outputs)))
    else:
      loss_op = tf.losses.softmax_cross_entropy(onehot_labels = tf.one_hot(dec_outputs, len(params['char2idx'])+1),
                                                logits = logits_or_ids,
                                                weights = tf.to_float(tf.sign(dec_outputs)),
                                                label_smoothing = params['label_smoothing'],)
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        decay_lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .99)
        
        train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': decay_lr,
                                           'sampling_proba': sampling_proba_fn()},
                                          every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)
      
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss_op)

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

In [0]:
def pad(test_strs):
  max_len = max([len(test_str) for test_str in test_strs])
  for test_str in test_strs:
    if len(test_str) < max_len:
      test_str += ['<pad>'] * (max_len - len(test_str))


def minimal_test(estimator):
  test_strs = [
    '你好',
    '早上好',
    '晚上好',
    '再见',
    '好久不见',
    '想死你了',
    '谢谢你',
    '爱你',
    '你好厉害啊',
    '你叫什么',
    '你几岁了',
    '现在几点',
    '今天天气怎么样',
    '我们现在在哪里',
    '讲个笑话',
    '你会几种语言呀',
    '你觉得我帅吗',
    '讨厌的周一',
    '好烦啊',
    '天气真好',
    '今天好冷',
    '今天好热',
    '下雨了',
    '风好大',
    '终于周五了',
    '我想去K歌',
  ]
  test_strs = [list(test_str) for test_str in test_strs]
  pad(test_strs)
  test_arrs = [[params['char2idx'].get(c, len(params['char2idx'])) for c in test_str] for test_str in test_strs]
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = np.asarray(test_arrs), shuffle = False)))
  predicted = np.asarray(predicted)
  print('-'*12)
  print('minimal test')
  for i, test_str in enumerate(test_strs):
    print('Q:', ' '.join([c for c in test_str if c != '<pad>']))
    for j in range(params['top_k']):
      sent = ' '.join([params['idx2char'].get(idx, '<unk>') for idx in predicted[i, :, j] if (idx!=0 and idx!=2)])
      print('A{}:'.format(j+1), sent)
    print()
  print('-'*12)

In [0]:
def is_increasing(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] >= history[i]:
      return False
  return True  

In [0]:
params = {
    'model_dir': '../model/lstm_seq2seq_scheduled',
    'log_path': '../log/lstm_seq2seq_scheduled.txt',
    'train_paths': ['../data/core.txt'] + ['../data/reduced.txt'],
    'test_paths': ['../data/core.txt'],
    'vocab_path': '../vocab/char.txt',
    'max_len': 30,
    'dropout_rate': .2,
    'rnn_units': 300,
    'activation': tf.nn.relu,
    'dec_layers': 1,
    'beam_width': 5,
    'top_k': 3,
    'length_penalty': .6,
    'coverage_penalty': .0,
    'label_smoothing': .3,
    'lr': 7e-4,
    'clip_norm': .1,
    'buffer_size': 180000,
    'batch_size': 64,
    'num_patience': 5,
}

In [0]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

In [16]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(
    save_checkpoints_steps=params['buffer_size']//params['batch_size'], keep_checkpoint_max=1),
  params=params)

best_ppl = 10000.
history_ppl = []
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)
  
  loss = estimator.evaluate(input_fn=lambda: dataset(is_training=False, params=params))['loss']
  ppl = np.exp(loss)
  logger.info("Perplexity: {:.3f}".format(ppl))
  history_ppl.append(ppl)

  if ppl < best_ppl:
    best_ppl = ppl
  logger.info("Best Perplexity: {:.3f}".format(best_ppl))

  if len(history_ppl) > params['num_patience'] and is_increasing(history_ppl):
    logger.info("Perplexity not improved over {} epochs, Early Stop".format(params['num_patience']))
    break
  
  # BLEU
  labels = [label for _, (_, label) in dataset(is_training=False, params=params)]
  labels = [j for i in labels for j in i.numpy()]
  labels = [[params['idx2char'].get(idx, '<unk>') for idx in arr if (idx!=0 and idx!=2)] for arr in labels]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))
  assert len(labels) == len(preds)
  preds = [[params['idx2char'].get(idx, '<unk>') for idx in arr[:, 0] if (idx!=0 and idx!=2)] for arr in preds]

  logger.info("BLEU: {:.1f}".format(tx.evals.corpus_bleu_moses(list_of_references=[[l] for l in labels], hypotheses=preds)))

INFO:tensorflow:Using config: {'_model_dir': '../model/lstm_seq2seq_scheduled', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2812, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efc70186588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initi

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ../model/lstm_seq2seq_scheduled/model.ckpt.
Reading ../data/core.txt
Reading ../data/reduced.txt
INFO:tensorflow:loss = 8.664225, step = 0
INFO:tensorflow:lr = 0.0007, sampling_proba = 0.050000012
INFO:tensorflow:global_step/sec: 7.85503
INFO:tensorflow:loss = 6.741941, step = 100 (12.736 sec)
INFO:tensorflow:lr = 0.00069929677, sampling_proba = 0.050954342 (12.736 sec)
INFO:tensorflow:global_step/sec: 8.12049
INFO:tensorflow:loss = 6.3732944, step = 200 (12.314 sec)
INFO:tensorflow:lr = 0.0006985943, sampling_proba = 0.05190766 (12.313 sec)
INFO:tensorflow:global_step/sec: 8.23214
INFO:tensorflow:loss = 6.359363, step = 300 (12.148 sec)
INFO:tensorflow:lr = 0.00069789257, sampling_proba = 0.05286008 (12.149 sec)
INFO:tensorflow:global_step/

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_scheduled/model.ckpt-2745
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2745 into ../model/lstm_seq2seq_scheduled/model.ckpt.
Reading ../data/core.txt
Reading ../data/reduced.txt
INFO:tensorflow:loss = 5.8926697, step = 2745
INFO:tensorflow:lr = 0.00068095216, sampling_proba = 0.075850606
INFO:tensorflow:global_step/sec: 7.70644
INFO:tensorflow:loss = 5.7774496, step = 2845 (12.978 sec)
INFO:tensorflow:lr = 0.00068026816, sampling_proba = 0.07677889 (12.975 sec)
INFO:tensorflow:global_step/sec: 8.14011
INFO:tensorflow:loss = 5.6262403, step = 2945 (12.284 sec)
INFO:tensorflow:lr = 0.0006795848, sampling_proba = 0.07770628 (12.284 sec)
INFO:tensorflow:global_step

...

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_scheduled/model.ckpt-101565
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 101565 into ../model/lstm_seq2seq_scheduled/model.ckpt.
Reading ../data/core.txt
Reading ../data/reduced.txt
INFO:tensorflow:loss = 5.7975173, step = 101565
INFO:tensorflow:lr = 0.00025222436, sampling_proba = 0.5
INFO:tensorflow:global_step/sec: 7.54726
INFO:tensorflow:loss = 5.8478456, step = 101665 (13.255 sec)
INFO:tensorflow:lr = 0.00025197095, sampling_proba = 0.5 (13.255 sec)
INFO:tensorflow:global_step/sec: 7.86944
INFO:tensorflow:loss = 5.691367, step = 101765 (12.709 sec)
INFO:tensorflow:lr = 0.00025171784, sampling_proba = 0.5 (12.709 sec)
INFO:tensorflow:global_step/sec: 7.96297
INFO:tensorflow:loss = 5.751156, step = 101865 (12.556 sec)
INFO:ten

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_scheduled/model.ckpt-104310
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 104310 into ../model/lstm_seq2seq_scheduled/model.ckpt.
Reading ../data/core.txt
Reading ../data/reduced.txt
INFO:tensorflow:loss = 5.8598456, step = 104310
INFO:tensorflow:lr = 0.000245361, sampling_proba = 0.5
INFO:tensorflow:global_step/sec: 7.54831
INFO:tensorflow:loss = 5.6617746, step = 104410 (13.249 sec)
INFO:tensorflow:lr = 0.00024511453, sampling_proba = 0.5 (13.247 sec)
INFO:tensorflow:global_step/sec: 7.97059
INFO:tensorflow:loss = 5.712252, step = 104510 (12.550 sec)
INFO:tensorflow:lr = 0.00024486834, sampling_proba = 0.5 (12.549 sec)
INFO:tensorflow:global_step/sec: 7.98528
INFO:tensorflow:loss = 5.6882243, step = 104610 (12.521 sec)
INFO:tens

KeyboardInterrupt: ignored